# Genomic Model Wrappers Usage Guide

This notebook provides comprehensive examples of how to use each wrapper in the genebeddings package.
All wrappers implement the standardized `BaseWrapper` API for consistent usage across different models.

## Table of Contents
1. [BaseWrapper API Overview](#BaseWrapper-API-Overview)
2. [NTWrapper (Nucleotide Transformer)](#NTWrapper)
3. [CaduceusWrapper](#CaduceusWrapper)
4. [BorzoiWrapper](#BorzoiWrapper)
5. [SpeciesLMWrapper](#SpeciesLMWrapper)
6. [RiNALMoWrapper](#RiNALMoWrapper)
7. [ConformerWrapper](#ConformerWrapper)
8. [ConvNovaWrapper](#ConvNovaWrapper)
9. [Advanced Usage Examples](#Advanced-Usage)
10. [Dependency Map Examples](#Dependency-Maps)

In [ ]:
# Common imports
import numpy as np
import torch
import matplotlib.pyplot as plt
from typing import List, Dict

# Test sequence - a realistic genomic sequence
TEST_SEQUENCE = "ATCGATCGATCGATCGATCGATCGATCGAAAAAAAAAAATTTTTTTTTTTCCCCCCCCCCGGGGGGGGGGAACCTTGGAACCTTGGAACCTTGGAACCTTGG" * 10
SHORT_SEQUENCE = "ATCGATCGATCGATCGATCGATCG"

print(f"Test sequence length: {len(TEST_SEQUENCE)}")
print(f"Short sequence: {SHORT_SEQUENCE}")

## BaseWrapper API Overview

All wrappers implement the same standard interface:

### Required Methods
- **`embed(seq, pool='mean', return_numpy=True)`**: Generate sequence embeddings

### Optional Methods (model-dependent)
- **`predict_nucleotides(seq, positions)`**: Get nucleotide probabilities at masked positions
- **`predict_tracks(seq)`**: Get genomic track predictions (Borzoi-style)

### Utility Methods
- **`get_capabilities()`**: List supported features
- **`supports_capability(name)`**: Check if feature is supported

In [ ]:
def demo_wrapper_capabilities(wrapper, name):
    """Demonstrate basic capabilities of any wrapper."""
    print(f"\n=== {name} Capabilities ===")
    print(f"Supported features: {wrapper.get_capabilities()}")
    
    # Test embedding
    if wrapper.supports_capability('embed'):
        print("\n📊 Testing embedding functionality:")
        
        # Mean pooled embedding
        emb_mean = wrapper.embed(SHORT_SEQUENCE, pool='mean')
        print(f"  Mean pooled: {emb_mean.shape}")
        
        # Token embeddings
        emb_tokens = wrapper.embed(SHORT_SEQUENCE, pool='tokens')
        print(f"  Token level: {emb_tokens.shape}")
        
        # CLS embedding (if supported)
        try:
            emb_cls = wrapper.embed(SHORT_SEQUENCE, pool='cls')
            print(f"  CLS token: {emb_cls.shape}")
        except:
            print(f"  CLS token: Not supported")
    
    # Test nucleotide prediction
    if wrapper.supports_capability('predict_nucleotides'):
        print("\n🧬 Testing nucleotide prediction:")
        try:
            # Create a sequence with masked positions
            masked_seq = SHORT_SEQUENCE[:10] + 'N' + SHORT_SEQUENCE[11:]
            probs = wrapper.predict_nucleotides(masked_seq, positions=None)
            print(f"  Predictions at masked positions: {len(probs)} positions")
            if len(probs) > 0:
                print(f"  Sample prediction: {probs[0]}")
        except Exception as e:
            print(f"  Error: {e}")
    
    # Test track prediction
    if wrapper.supports_capability('predict_tracks'):
        print("\n📈 Testing track prediction:")
        try:
            tracks = wrapper.predict_tracks(SHORT_SEQUENCE)
            print(f"  Track predictions: {tracks.shape}")
        except Exception as e:
            print(f"  Error: {e}")
    
    print(f"\n✅ {name} demo completed")

## NTWrapper (Nucleotide Transformer)

**Model**: Nucleotide Transformer v2 (500M parameters)  
**Source**: HuggingFace (`InstaDeepAI/nucleotide-transformer-v2-500m-multi-species`)  
**Tokenization**: k-mer based (typically k=6, non-overlapping)  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: Large pre-trained model, good for sequence understanding

In [ ]:
try:
    from nt_wrapper import NTWrapper
    
    print("🔄 Loading NTWrapper...")
    nt_model = NTWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(nt_model, "NTWrapper")
    
    print("\n🔬 NTWrapper-specific features:")
    
    # Test k-mer tokenization
    tokens = nt_model.tokens_from_seq(SHORT_SEQUENCE)
    print(f"K-mer tokens: {tokens.shape}")
    print(f"Inferred k-mer size: {nt_model.k}")
    
    # Test batch embeddings
    sequences = [SHORT_SEQUENCE, SHORT_SEQUENCE[:15], SHORT_SEQUENCE + "AAAA"]
    batch_embeddings = [nt_model.embed(seq, pool='mean') for seq in sequences]
    print(f"Batch embeddings: {len(batch_embeddings)} sequences")
    
    # Test pattern matching (legacy feature)
    try:
        # Get logits for a masked position
        tokens_no_specials = nt_model.tokens_from_seq(SHORT_SEQUENCE)
        logits = nt_model.get_masked_position_logits(tokens_no_specials, 0)
        print(f"Logits shape: {logits.shape}")
        
        # Pattern matching
        pattern_result = nt_model.pattern_matching_filter(logits, "ATCGNN")
        print(f"Pattern matching result: {pattern_result}")
    except Exception as e:
        print(f"Pattern matching error: {e}")
    
except ImportError:
    print("❌ NTWrapper not available (missing dependencies)")
except Exception as e:
    print(f"❌ NTWrapper error: {e}")

## CaduceusWrapper

**Model**: Caduceus (BiDirectional Mamba for DNA)  
**Source**: HuggingFace (`kuleshov-group/caduceus-*`)  
**Tokenization**: Character-level or k-mer (often k=1)  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: Long sequence support (~131k tokens), bidirectional architecture

In [ ]:
try:
    from caduceus_wrapper import CaduceusWrapperOld as CaduceusWrapper
    
    print("🔄 Loading CaduceusWrapper...")
    caduceus_model = CaduceusWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(caduceus_model, "CaduceusWrapper")
    
    print("\n🔬 CaduceusWrapper-specific features:")
    
    # Test with long sequence
    long_sequence = TEST_SEQUENCE[:1000]  # 1k bp test
    long_emb = caduceus_model.embed(long_sequence, pool='mean')
    print(f"Long sequence embedding: {long_emb.shape}")
    
    # Test character-level tokenization
    tokens = caduceus_model.tokens_from_seq(SHORT_SEQUENCE)
    print(f"Tokens shape: {tokens.shape}")
    print(f"Inferred k-mer size: {caduceus_model.k}")
    
    # Test nucleotide prediction with multiple positions
    masked_seq = "ATCGNNNGATCG"
    predictions = caduceus_model.predict_nucleotides(masked_seq, positions=None)
    print(f"Multi-position predictions: {len(predictions)} positions")
    
except ImportError:
    print("❌ CaduceusWrapper not available (missing dependencies)")
except Exception as e:
    print(f"❌ CaduceusWrapper error: {e}")

## BorzoiWrapper

**Model**: Borzoi (Genomic Track Prediction)  
**Source**: HuggingFace (`johahi/flashzoi-replicate-0`)  
**Tokenization**: One-hot encoding (A/C/G/T → 4 channels)  
**Capabilities**: Embeddings, genomic track prediction  
**Strengths**: Predicts chromatin accessibility, TF binding, etc.

In [ ]:
try:
    from borzoi_wrapper import BorzoiWrapper
    
    print("🔄 Loading BorzoiWrapper...")
    borzoi_model = BorzoiWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(borzoi_model, "BorzoiWrapper")
    
    print("\n🔬 BorzoiWrapper-specific features:")
    
    # Borzoi requires specific input length (524,288 bp by default)
    print(f"Required input length: {borzoi_model.input_len} bp")
    print(f"Target output length: {borzoi_model.target_len} positions")
    
    # Test with padded sequence
    test_seq = TEST_SEQUENCE[:10000]  # Use first 10k bp
    
    # Get embeddings
    embedding = borzoi_model.embed(test_seq, pool='mean')
    print(f"Embedding shape: {embedding.shape}")
    
    # Get track predictions
    tracks = borzoi_model.predict_tracks(test_seq)
    print(f"Track predictions shape: {tracks.shape}")
    print(f"Number of tracks: {tracks.shape[0]}")
    print(f"Number of positions: {tracks.shape[1]}")
    
    # Visualize some tracks
    plt.figure(figsize=(12, 6))
    for i in range(min(5, tracks.shape[0])):
        plt.subplot(2, 3, i+1)
        plt.plot(tracks[i, :100])  # First 100 positions
        plt.title(f'Track {i}')
        plt.xlabel('Position')
        plt.ylabel('Signal')
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("❌ BorzoiWrapper not available (missing borzoi_pytorch package)")
except Exception as e:
    print(f"❌ BorzoiWrapper error: {e}")

## SpeciesLMWrapper

**Model**: SpeciesLM (Species-aware Language Model)  
**Source**: HuggingFace (`gagneurlab/SpeciesLM`)  
**Tokenization**: k=6 overlapping k-mers (stride=1)  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: Species conditioning, automatic chunking for long sequences

In [ ]:
try:
    from specieslm_wrapper import SpeciesLMWrapper
    
    print("🔄 Loading SpeciesLMWrapper...")
    specieslm_model = SpeciesLMWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(specieslm_model, "SpeciesLMWrapper")
    
    print("\n🔬 SpeciesLMWrapper-specific features:")
    
    print(f"Max tokens per chunk: {specieslm_model.max_model_tokens}")
    print(f"Window size: ~{specieslm_model.window_size} nucleotides")
    
    # Test chunking with long sequence
    long_seq = TEST_SEQUENCE[:2000]  # 2k bp sequence
    chunks = specieslm_model._chunk_sequence(long_seq)
    print(f"Chunked {len(long_seq)} bp into {len(chunks)} chunks")
    
    # Test embedding with chunking
    long_emb = specieslm_model.embed(long_seq, pool='mean')
    print(f"Long sequence embedding: {long_emb.shape}")
    
    # Test full sequence probability matrix
    short_seq = SHORT_SEQUENCE[:50]  # Use shorter sequence for demo
    prob_matrix = specieslm_model.acgt_probs(short_seq)
    print(f"Probability matrix: {prob_matrix.shape}")
    
    # Visualize probability matrix
    plt.figure(figsize=(12, 4))
    plt.imshow(prob_matrix.T, aspect='auto', cmap='viridis')
    plt.colorbar(label='Probability')
    plt.xlabel('Position')
    plt.ylabel('Nucleotide')
    plt.yticks([0, 1, 2, 3], ['A', 'C', 'G', 'T'])
    plt.title('Nucleotide Probabilities Across Sequence')
    plt.show()
    
    # Test species conditioning
    try:
        species_emb = specieslm_model.embed(SHORT_SEQUENCE, pool='mean', species_proxy='human')
        print(f"Species-conditioned embedding: {species_emb.shape}")
    except:
        print("Species conditioning not available")
    
except ImportError:
    print("❌ SpeciesLMWrapper not available (missing dependencies)")
except Exception as e:
    print(f"❌ SpeciesLMWrapper error: {e}")

## RiNALMoWrapper

**Model**: RiNALMo (RNA Language Model)  
**Source**: Pretrained via `rinalmo` package  
**Tokenization**: Character-level (k=1)  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: RNA-specialized, handles T↔U conversion

In [ ]:
try:
    from rinalmo_wrapper import RiNALMoWrapper
    
    print("🔄 Loading RiNALMoWrapper...")
    rinalmo_model = RiNALMoWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(rinalmo_model, "RiNALMoWrapper")
    
    print("\n🔬 RiNALMoWrapper-specific features:")
    
    # Test RNA vs DNA handling
    dna_seq = "ATCGATCG"
    rna_seq = "AUCGAUCG"
    
    dna_emb = rinalmo_model.embed(dna_seq, pool='mean')
    rna_emb = rinalmo_model.embed(rna_seq, pool='mean')
    
    print(f"DNA embedding: {dna_emb.shape}")
    print(f"RNA embedding: {rna_emb.shape}")
    
    # Check if model uses RNA vocabulary
    print(f"Model uses RNA vocab: {rinalmo_model.model_uses_rna_vocab}")
    print(f"DNA output enabled: {rinalmo_model.dna_output}")
    
    # Test batch processing
    sequences = [dna_seq, rna_seq, "AAAUUUGGCCCC"]
    batch_embeddings = [rinalmo_model.embed(seq, pool='mean') for seq in sequences]
    print(f"Batch processing: {len(batch_embeddings)} sequences")
    
    # Test nucleotide prediction
    masked_seq = "ATCGNGATCG"
    predictions = rinalmo_model.predict_nucleotides(masked_seq, positions=None)
    print(f"Nucleotide predictions: {len(predictions)} positions")
    
except ImportError:
    print("❌ RiNALMoWrapper not available (missing rinalmo package)")
except Exception as e:
    print(f"❌ RiNALMoWrapper error: {e}")

## ConformerWrapper

**Model**: Conformer-based genomic model (base-anchored)  
**Source**: Local model files (required)  
**Tokenization**: k=1 with base-anchored GenomicBPE  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: Local deployment, base-anchored tokenization

In [ ]:
try:
    from conformer_wrapper import ConformerWrapper
    
    # ConformerWrapper requires local model files
    # You'll need to provide paths to your local model
    tokenizer_path = "/path/to/your/tokenizer.tkz"
    checkpoint_path = "/path/to/your/model.pt"
    
    print("🔄 Loading ConformerWrapper...")
    print("⚠️  Note: Requires local model files")
    
    # Uncomment and modify paths if you have local model files
    """
    conformer_model = ConformerWrapper(
        tokenizer_path=tokenizer_path,
        checkpoint_path=checkpoint_path
    )
    
    # Basic capabilities demo
    demo_wrapper_capabilities(conformer_model, "ConformerWrapper")
    
    print("\n🔬 ConformerWrapper-specific features:")
    
    # Test base-anchored tokenization
    print(f"Base-anchored tokenization (k=1): {conformer_model.k}")
    
    # Test DataParallel support
    if torch.cuda.device_count() > 1:
        print(f"Multi-GPU support: {torch.cuda.device_count()} GPUs available")
    """
    
    print("📝 To use ConformerWrapper:")
    print("   1. Obtain tokenizer (.tkz) and checkpoint (.pt) files")
    print("   2. Install genomenet dependencies (anchor_tokenizer, conformer)")
    print("   3. Update tokenizer_path and checkpoint_path variables above")
    print("   4. Uncomment the code block")
    
except ImportError:
    print("❌ ConformerWrapper not available (missing genomenet dependencies)")
except Exception as e:
    print(f"❌ ConformerWrapper error: {e}")

## ConvNovaWrapper

**Model**: ConvNova (Dilated CNN model)  
**Source**: Optional local checkpoint  
**Tokenization**: k=1 base-level  
**Capabilities**: Embeddings, nucleotide prediction  
**Strengths**: Can work without checkpoint, efficient CNN architecture

In [ ]:
try:
    from convnova_wrapper import ConvNovaWrapper
    
    print("🔄 Loading ConvNovaWrapper...")
    # Can work without checkpoint (uses random initialization)
    convnova_model = ConvNovaWrapper()
    
    # Basic capabilities demo
    demo_wrapper_capabilities(convnova_model, "ConvNovaWrapper")
    
    print("\n🔬 ConvNovaWrapper-specific features:")
    
    # Test base-level tokenization
    tokens = convnova_model.tokens_from_seq(SHORT_SEQUENCE)
    print(f"Base-level tokens: {tokens.shape}")
    print(f"Token k-mer size: {convnova_model.k}")
    
    # Test CNN architecture info
    print(f"Model architecture: Dilated CNN")
    
    # Test with longer sequence
    longer_seq = TEST_SEQUENCE[:500]
    longer_emb = convnova_model.embed(longer_seq, pool='mean')
    print(f"Longer sequence embedding: {longer_emb.shape}")
    
    # Test nucleotide prediction
    masked_seq = "ATCGNNNGATCG"
    predictions = convnova_model.predict_nucleotides(masked_seq, positions=None)
    print(f"Predictions: {len(predictions)} masked positions")
    
    # Note about checkpoints
    print("\n📝 Note: Currently using random initialization")
    print("   For trained model, provide checkpoint_path parameter")
    
except ImportError:
    print("❌ ConvNovaWrapper not available (missing dependencies)")
except Exception as e:
    print(f"❌ ConvNovaWrapper error: {e}")

## Advanced Usage Examples

Here are some advanced patterns for using the wrappers effectively.

In [ ]:
def compare_embeddings(models_dict, sequence):
    """Compare embeddings across different models."""
    embeddings = {}
    
    print(f"🔬 Comparing embeddings for sequence: {sequence[:30]}...")
    
    for name, model in models_dict.items():
        try:
            if model.supports_capability('embed'):
                emb = model.embed(sequence, pool='mean')
                embeddings[name] = emb
                print(f"  {name}: {emb.shape}, mean={emb.mean():.4f}, std={emb.std():.4f}")
        except Exception as e:
            print(f"  {name}: Error - {e}")
    
    # Compute pairwise similarities
    if len(embeddings) > 1:
        print("\n📊 Pairwise cosine similarities:")
        names = list(embeddings.keys())
        for i, name1 in enumerate(names):
            for j, name2 in enumerate(names[i+1:], i+1):
                emb1, emb2 = embeddings[name1], embeddings[name2]
                # Ensure same length for comparison
                min_len = min(len(emb1), len(emb2))
                similarity = np.dot(emb1[:min_len], emb2[:min_len]) / (np.linalg.norm(emb1[:min_len]) * np.linalg.norm(emb2[:min_len]))
                print(f"  {name1} vs {name2}: {similarity:.4f}")
    
    return embeddings

def benchmark_prediction_accuracy(models_dict, test_sequences):
    """Benchmark nucleotide prediction accuracy."""
    print("🎯 Benchmarking nucleotide prediction accuracy...")
    
    results = {}
    
    for name, model in models_dict.items():
        if not model.supports_capability('predict_nucleotides'):
            continue
            
        accuracies = []
        
        for seq in test_sequences:
            # Mask random position
            mask_pos = len(seq) // 2
            true_base = seq[mask_pos]
            masked_seq = seq[:mask_pos] + 'N' + seq[mask_pos+1:]
            
            try:
                predictions = model.predict_nucleotides(masked_seq, positions=None)
                if len(predictions) > 0:
                    pred_dict = predictions[0]
                    predicted_base = max(pred_dict.keys(), key=lambda k: pred_dict[k])
                    accuracies.append(1.0 if predicted_base == true_base else 0.0)
            except:
                continue
        
        if accuracies:
            avg_accuracy = np.mean(accuracies)
            results[name] = avg_accuracy
            print(f"  {name}: {avg_accuracy:.3f} accuracy ({len(accuracies)} tests)")
    
    return results

In [ ]:
# Example: Compare available models
available_models = {}

# Add models that were successfully loaded above
# Note: In practice, you'd check which models are available

try:
    if 'nt_model' in locals():
        available_models['NT'] = nt_model
except:
    pass

try:
    if 'caduceus_model' in locals():
        available_models['Caduceus'] = caduceus_model
except:
    pass

try:
    if 'specieslm_model' in locals():
        available_models['SpeciesLM'] = specieslm_model
except:
    pass

try:
    if 'convnova_model' in locals():
        available_models['ConvNova'] = convnova_model
except:
    pass

print(f"Available models for comparison: {list(available_models.keys())}")

if available_models:
    # Compare embeddings
    test_seq = "ATCGATCGATCGATCGATCGATCGATCG"
    embeddings = compare_embeddings(available_models, test_seq)
    
    # Benchmark prediction accuracy
    test_sequences = [
        "ATCGATCGATCGATCG",
        "AAATTTGGGCCCAAATTT", 
        "GCTAGCTAGCTAGCTA"
    ]
    accuracies = benchmark_prediction_accuracy(available_models, test_sequences)
else:
    print("❌ No models available for comparison")

## Dependency Map Examples

Demonstrate the dependency map functionality with the available models.

In [ ]:
try:
    from dependency_map import compute_dependency_map, plot_dependency_map
    
    print("🗺️  Testing dependency map functionality...")
    
    # Use a model that supports both embedding and nucleotide prediction
    test_model = None
    model_name = "None"
    
    for name, model in available_models.items():
        if model.supports_capability('embed') and model.supports_capability('predict_nucleotides'):
            test_model = model
            model_name = name
            break
    
    if test_model is None:
        # Fall back to embedding-only model
        for name, model in available_models.items():
            if model.supports_capability('embed'):
                test_model = model
                model_name = name
                break
    
    if test_model is not None:
        print(f"Using {model_name} for dependency map demo")
        
        # Test sequence - use a shorter sequence for demo
        dep_test_seq = "ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG"  # 42 bp
        
        # Method 1: Mutual information (works with embedding-only models)
        print("\n🔬 Computing mutual information dependency map...")
        dep_mutual = compute_dependency_map(
            dep_test_seq,
            test_model,
            method="mutual_info",
            range_start=5,
            range_end=25,  # 20x20 matrix
            embedding_distance="euclidean"
        )
        
        print(f"Dependency matrix shape: {dep_mutual.shape}")
        print(f"Value range: [{dep_mutual.min():.4f}, {dep_mutual.max():.4f}]")
        
        # Plot the dependency map
        fig = plot_dependency_map(
            dep_mutual,
            sequence=dep_test_seq[5:25],  # Corresponding sequence region
            title=f"Mutual Information Dependency Map ({model_name})"
        )
        plt.show()
        
        # Method 2: Epistatic interactions (if model supports both capabilities)
        if test_model.supports_capability('predict_nucleotides'):
            print("\n🧬 Computing epistatic dependency map...")
            
            # Use smaller region for computational efficiency
            dep_epistatic = compute_dependency_map(
                dep_test_seq,
                test_model,
                method="epistatic",
                range_start=10,
                range_end=20,  # 10x10 matrix
                embedding_distance="euclidean"
            )
            
            print(f"Epistatic matrix shape: {dep_epistatic.shape}")
            print(f"Value range: [{dep_epistatic.min():.4f}, {dep_epistatic.max():.4f}]")
            
            fig = plot_dependency_map(
                dep_epistatic,
                sequence=dep_test_seq[10:20],
                title=f"Epistatic Dependency Map ({model_name})"
            )
            plt.show()
        
        # Method 3: Log probabilities (if model supports nucleotide prediction)
        if test_model.supports_capability('predict_nucleotides'):
            print("\n📊 Computing logprobs dependency map...")
            
            dep_logprobs = compute_dependency_map(
                dep_test_seq,
                test_model,
                method="logprobs",
                range_start=12,
                range_end=20,  # 8x8 matrix
            )
            
            print(f"Logprobs matrix shape: {dep_logprobs.shape}")
            print(f"Value range: [{dep_logprobs.min():.4f}, {dep_logprobs.max():.4f}]")
            
            fig = plot_dependency_map(
                dep_logprobs,
                sequence=dep_test_seq[12:20],
                title=f"Log-Probability Dependency Map ({model_name})"
            )
            plt.show()
    
    else:
        print("❌ No suitable model available for dependency map demo")
        
except ImportError:
    print("❌ Dependency map module not available")
except Exception as e:
    print(f"❌ Dependency map error: {e}")

## Summary and Best Practices

### Model Selection Guide

| Use Case | Recommended Model(s) | Notes |
|----------|----------------------|-------|
| **General sequence embedding** | NTWrapper, CaduceusWrapper | Large pretrained models |
| **Long sequences (>100k bp)** | CaduceusWrapper | Supports ~131k tokens |
| **Nucleotide prediction** | NTWrapper, SpeciesLMWrapper | Strong MLM capabilities |
| **Genomic tracks/signals** | BorzoiWrapper | Specialized for chromatin/TF prediction |
| **Species-specific analysis** | SpeciesLMWrapper | Built-in species conditioning |
| **RNA sequences** | RiNALMoWrapper | Handles T↔U conversion |
| **Local/custom models** | ConformerWrapper, ConvNovaWrapper | For proprietary models |

### Performance Tips

1. **Batch processing**: Use list comprehensions for multiple sequences
2. **Memory management**: Use appropriate pooling (`mean` vs `tokens`)
3. **GPU utilization**: Models automatically use available GPUs
4. **Sequence length**: Check model-specific limits and chunking behavior

### Common Patterns

```python
# Pattern 1: Check capabilities before use
if model.supports_capability('predict_nucleotides'):
    predictions = model.predict_nucleotides(masked_seq, positions=None)

# Pattern 2: Handle different pooling strategies
global_emb = model.embed(seq, pool='mean')      # Global representation
position_embs = model.embed(seq, pool='tokens') # Position-wise embeddings

# Pattern 3: Robust error handling
try:
    result = model.predict_nucleotides(seq, positions=None)
except NotImplementedError:
    print(f"Model doesn't support nucleotide prediction")
```

This notebook provides a comprehensive overview of all wrapper functionality. Each wrapper implements the same standardized API while exposing model-specific capabilities and optimizations.